In [ ]:
pip install openai

In [7]:
import os
import openai
import time
import re

def get_api_key() -> str:
    '''Returns openai API key from API folder'''
    if os.path.exists('api/openai.txt'):
        with open('api/openai.txt') as f:
            return f.read().strip()
    return ''

def validate_key() -> bool:
    '''Checks if openai API key is valid'''
    openai.api_key = get_api_key()
    try:
        openai.Completion.create(engine="davinci", prompt="This is a test", max_tokens=5)
        return True
    except Exception as e:
        print('errors', 'GPT-3 API', str(e))
        return False
    
def get_response(messages):
    '''Gets response from openai API'''
    #print(messages)
    response = None
    for i in range(3):
        if not validate_key():
            break
        response = query_gpt(messages)
        if response:
            break
    
    if not response:
        print('errors', 'GPT-3 API', 'Max connection attempts reached. Check internet connection.')
        return 'Sorry. My dog to human translator is unable to connect to the internet, right now.'
    
    return response['choices'][0]['message']['content']

def query_gpt(messages):
    '''Tries to get ChatGPT to generate a text response to messages
    if it fails, it waits 1 second and tries again.'''
    try:
        response = openai.ChatCompletion.create(
            model='gpt-3.5-turbo',
            messages=messages,
            max_tokens=100,
        )
        return response
    
    except Exception as e:
        print('errors', 'GPT-3 API', str(e))
        time.sleep(1)
        return None

def format_question(question):
    '''Formats a string as a user prompt for ChatGPT'''
    return [{
        'role':'user',
        'content': question,
    }]

def format_instructions(instructions):
    '''Formats a string as a system instruction for ChatGPT'''
    return [{
        'role':'system',
        'content': instructions,
    }]

def read_from_file(filepath):
    '''Reads all data from a file at filepath'''
    with open(filepath, 'r', encoding='utf-8') as infile:
        return infile.read()

def write_to_file(filepath, content):
    '''Writes all data in content to file at filepath'''
    with open(filepath, 'w', encoding='utf-8') as outfile:
        outfile.write(content)

def validate_sentiment_score(score: str):
    '''Returns true if the sentiment score is a valid structure.'''
    valid_pattern = '^[\-]{0,1}[0-9]{1}[0]{0,1},[a-z]*( [a-z]*){0,2}(\n[\-]{0,1}[0-9]{1}[0]{0,1},[a-z]*( [a-z]*){0,2})*$'
    return bool(re.search(valid_pattern, score))

In [ ]:
def process_batch(instructions, batch, start):
    response_accepted = False
    while True:
        print('Processing comments', start, 'to', start + len(batch) - 1)
        prompt_text = ','.join(batch)
        prompt = instructions + format_question(prompt_text)
        response = get_response(prompt)
        if validate_sentiment_score(response) and response.split('\n') == len(batch):
            return response.split('\n')
        print('Invalid scores:', response)
        print('Generated invalid scores. Trying again...')

def get_sentiment_scores():
    '''Returns a new line separated list of sentiment scores for 
    each user feedback comment.'''
    if not validate_key():
        print('no key found.')
        return

    instructions = read_from_file('instructions.txt')
    inst_length = len(instructions.split(' '))
    instructions = format_instructions(instructions)
    comments = read_from_file('comments.txt')
    comments = comments.strip().split('\n')
    batch_size = 10
    
    scores = []
    for i in range(int(len(comments)/batch_size)+1):
        fine = False
        while not fine:
            if len(comments[i*batch_size:(i+1)*batch_size]) == 0:
                fine = True
                continue
            print('Getting', i*batch_size, 'thru', (i+1)*batch_size)
            prompt = instructions + format_question('\n'.join(comments[i*batch_size:(i+1)*batch_size]))
            response = get_response(prompt)
            current_scores = response.strip()
            print(current_scores)
            fine = validate_sentiment_score(current_scores)
            if not fine:
                print('Invalid score, trying again...')
                continue
            current_scores = current_scores.split('\n')
            if len(comments[i*batch_size:(i+1)*batch_size]) != len(current_scores):
                print('Same length?', len(comments[i*batch_size:(i+1)*batch_size]) != len(current_scores))
                print('Length of batch:', len(comments[i*batch_size:(i+1)*batch_size]))
                print('Score length:', len(current_scores))
                print('\n\n')
                fine = False
                continue    
            scores = scores + current_scores
    return scores
    
    
sentiment = get_sentiment_scores()

In [ ]:
write_to_file('sentiment.txt', '\n'.join(sentiment))